In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

**MARKDOWNS ARE ABOVE THE RESPECTIVE CODE**
* Read the file using : pd.read_csv(" "). And print first 5 rows.

In [ ]:
df = pd.read_csv("../input/kc_house_data.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

## Data Engineering & Feature Extraction


* Create a dataframe in which "features" is a column where all features of the dataset are present. 
* Another column "unique_count" shows respective unique values of the features.

In [ ]:
unique_df = pd.DataFrame(columns = ["features", "unique_count"])
for col in df.columns:
    unique_df = unique_df.append({"features": col, "unique_count" : len(df[col].unique())}, ignore_index = True)
unique_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 12))
corr = df.corr()
sns.heatmap(corr, annot = True,fmt='.2f',
    cmap=sns.diverging_palette(20, 220, n=200),square=True)

In [ ]:
print(pd.Series(df["bathrooms"]).value_counts())
print(pd.Series(df["floors"]).value_counts())
print(pd.Series(df["waterfront"]).value_counts())
print(pd.Series(df["view"]).value_counts())
print(pd.Series(df["grade"]).value_counts())

* Normalising the "grade" feature.

In [ ]:
from sklearn import preprocessing
x = df[["grade"]].values

min_max_scaler = preprocessing.MinMaxScaler()

x_scaled = min_max_scaler.fit_transform(x)

# Run the normalizer on the dataframe
df_normalized = pd.DataFrame(x_scaled)
df["grade"] = df_normalized

* Cleaning "view" feature : 1 if has been viewed else 0

In [ ]:
df["any_view"] = df["view"].apply(lambda x : 1 if x > 0 else 0)
df["no_of_views"] = df["view"]
df.drop(["view"], axis = 1, inplace = True)

* Droping rows with no of bedrooms more than 10 and equal to 0.

In [ ]:
df.drop(df[df["bedrooms"] > 10].index, inplace = True)
df.drop(df[df["bedrooms"] == 0].index, inplace = True)

* Cleaning the values of "bathrooms" and "floors" features

In [ ]:
#df["bathrooms"] = np.array(df["bathrooms"])
df["bathrooms"] = np.around(df["bathrooms"])
df["bathrooms"] = df["bathrooms"].astype(int)

df["floors"] = np.around(df["floors"])
df["floors"] = df["floors"].astype(int)
pd.Series(df["floors"]).value_counts()

* Creating a new feature as "basement" which indicates the presence of a basement.

In [ ]:
df["basement"] = df["sqft_basement"].apply(lambda x : 1 if x > 0 else 0)

* Firstly, convert the "date" feature in date-time format. Then extract the years to make a new feature as "yr_sold" in our dataframe.
* Make a new feature as "age_house" which shows the age of the house.

In [ ]:

df["date"] = pd.to_datetime(df["date"])
df["yr_sold"] = df["date"].dt.year

df["age_house"] = df["yr_sold"] - df["yr_built"]

* Adding a feature as "any_renovation" which indicates that is there any renovation happened.

In [ ]:
df["any_renovation"] = df["yr_renovated"].apply(lambda x : 1 if x > 0 else 0)

* Adding a feature as "sqft_living_change" which indicates whether there is a change in sqft living in 2015.

In [ ]:
df["sqft_living_change"] = (df["sqft_living15"] - df["sqft_living"])
df["sqft_living_change"] = df["sqft_living_change"].apply(lambda x : 0 if x == 0 else 1)
#df["sqft_living_change"]

In [ ]:
df["sqft_living_inc"] = (df["sqft_living15"] - df["sqft_living"])
df["sqft_living_inc"] = df["sqft_living_inc"].apply(lambda x : 0 if x < 0 else x)
df["sqft_living_inc"]

In [ ]:
df["sqft_living_dec"] = (df["sqft_living"] - df["sqft_living15"])
df["sqft_living_dec"] = df["sqft_living_dec"].apply(lambda x : 0 if x < 0 else x)
df["sqft_living_dec"]

In [ ]:
df.head()

In [ ]:
df["renovation_after_sold"] = (df["yr_sold"] - df["yr_renovated"]).apply(lambda x : 1 if x <= 0 else 0)
#df[df["renovation_after_sold"] == 1]

* Make scatter plots for the features having unique values more than 70 against "price" feature. 
* Make Boxplots for the features having unique values less than 70 against "price" feature.

In [ ]:
col_scatterplot = unique_df[unique_df["unique_count"] > 70]["features"]
col_boxplot = unique_df[unique_df["unique_count"] <= 70]["features"]

import matplotlib.pyplot as plt
for col in col_scatterplot:
    if col != "date":
        if col != "price": 
            plt.scatter(df[col], df["price"])
            plt.xlabel(col)
            plt.ylabel("Price")
            plt.show()

import seaborn as sns
for col in col_boxplot:
    if col != "price":
        sns.boxplot(df[col], df["price"])
        plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 12))
corr = df.corr()
sns.heatmap(corr, annot = True,fmt='.2f',
    cmap=sns.diverging_palette(20, 220, n=200),square=True)

In [ ]:
x = df[["basement","yr_renovated","bathrooms", "bedrooms", "sqft_living","sqft_basement", "any_view", "no_of_views", "grade", "sqft_above", "sqft_living15", "floors", "waterfront","lat", "sqft_living_dec", "any_renovation"]]
y = df["price"]

In [ ]:
from sklearn.model_selection import train_test_split as split_data
from sklearn.linear_model import LinearRegression

x_train, x_test, y_train, y_test = split_data(x, y, random_state = 0)
reg = LinearRegression()
reg.fit(x_train, y_train)
predicted = reg.predict(x_test)
from sklearn import metrics
r_square = metrics.r2_score(y_test, predicted)
print(r_square)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
regressor = GradientBoostingRegressor(n_estimators=500,max_depth=4,min_samples_split=2,learning_rate=0.05,loss='ls')
regressor.fit(x_train, y_train)
GBM_predicted_Values=regressor.predict(x_test)
print('Gradient Boosting Regression R-squared', round(regressor.score(x_test, y_test), 3))
    